## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [2]:
!pip install keras

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import keras
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np

Using TensorFlow backend.


In [18]:
batch_size = 64
num_classes = 404
epochs = 20

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

404 train samples
102 test samples


In [19]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(13,)))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                224       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 404)               6868      
Total params: 7,364
Trainable params: 7,364
Non-trainable params: 0
___________________

In [20]:
history = model.fit(x_train, y_train, epochs=epochs, validation_split=.1)
scores = model.evaluate(x_test,y_test)
print(f'{model.metric_names[1]}: {scores[1]*100}')

Instructions for updating:
Use tf.cast instead.
Train on 363 samples, validate on 41 samples
Epoch 1/20
363/363 [==============================] - 0s 696us/step - loss: 5.9747 - acc: 0.0523 - val_loss: 5.9408 - val_acc: 0.0244
Epoch 2/20
363/363 [==============================] - 0s 49us/step - loss: 5.8718 - acc: 0.0661 - val_loss: 5.8345 - val_acc: 0.0244
Epoch 3/20
363/363 [==============================] - 0s 39us/step - loss: 5.7378 - acc: 0.0716 - val_loss: 5.6920 - val_acc: 0.0244
Epoch 4/20
363/363 [==============================] - 0s 43us/step - loss: 5.5674 - acc: 0.0744 - val_loss: 5.4939 - val_acc: 0.0244
Epoch 5/20
363/363 [==============================] - 0s 40us/step - loss: 5.3176 - acc: 0.0744 - val_loss: 5.2202 - val_acc: 0.0244
Epoch 6/20
363/363 [==============================] - 0s 44us/step - loss: 4.9871 - acc: 0.0716 - val_loss: 4.8646 - val_acc: 0.0244
Epoch 7/20
363/363 [==============================] - 0s 41us/step - loss: 4.5967 - acc: 0.0634 - val_loss: 

AttributeError: 'Sequential' object has no attribute 'metric_names'

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [0]:
##### Your Code Here #####

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.